<a href="https://colab.research.google.com/github/Juddt/Projet-Python-DAVROUX-DIDENOT/blob/quant_a/20hPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q streamlit yfinance pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 72.4 MB/s eta 0:00:00


In [40]:
%%writefile quantA.py

#imports
#web app
import streamlit as st
import pandas as pd
import numpy as np
#financial data
import yfinance as yf
import matplotlib.pyplot as plt
# display local time
from datetime import datetime
import pytz

time_zone_france = pytz.timezone("Europe/Paris")

#configuration of strealit
tittle= " Single Asset Analysis Module"
page="wide"

st.set_page_config(
    page_title=tittle,
    layout=page)

#auto refresh every 5 minutes, wich corresponds to 300 seconds
st.markdown(
    "<meta http-equiv='refresh' content='300'>",
    unsafe_allow_html=True)

#timeframe for yfinance settings
different_timeframes = { "1d":  {"period": "1d",  "interval": "5m"},
    "5d":{"period":"5d",  "interval": "15m"},
    "1m":{"period":"1mo", "interval": "1h"},
    "6m":{"period":"6mo", "interval": "1d"},
    "1y":{"period":"1y",  "interval": "1d"},
    "5y":{"period":"5y",  "interval": "1wk"},
    "Max":{"period":"max", "interval": "1wk"},}

# default sma values for each timeframe to keep consistency
sma_values = {"1d":  {"fast": 9,  "slow": 21},
    "5d":{"fast": 10,"slow": 50},
    "1m": {"fast": 20,"slow": 50},
    "6m": {"fast": 50,"slow": 100},
    "1y": {"fast": 50,"slow": 150},
    "5y": {"fast": 50,"slow": 200},
    "Max": {"fast": 50,"slow": 200},}

#default rsi window for each timeframe
window_rsi = {"1d": 7,
    "5d": 10,
    "1m":14,
    "6m":14,
    "1y":21,
    "5y": 21,
    "Max":21}

#number of days for each timeframe
number_days = {"1d": 1,
    "5d":5,
    "1m":30,
    "6m":180,
    "1y":365,
    "5y": 5 * 365}

strategies_labels = {
    "Buy & Hold": "Buy & Hold",
    "SMA Crossover": "SMA Crossover",
    "SMA + RSI": "SMA + RSI",
    "Simple Momentum": "Simple Momentum",
}

# users parameters on the sidebar

#tittle
st.sidebar.header(" User Settings")
st.sidebar.caption(  "1️ - Choose an asset ")
st.sidebar.caption( "2 - Set initial capital")
st.sidebar.caption("3 - Select a timeframe ")
st.sidebar.caption(" 4 - Pick a strategy")
st.sidebar.caption("5- Adjust parameters")

#inputs to choose the ticker, the initial capital
ticker= st.sidebar.text_input( "Ticker", value="AAPL")

capital = st.sidebar.number_input("Initial capital  in $", 10, 100000, 100, step=1)

#timeframe selection in our selection of diffeent timeframes
timeframe= st.sidebar.radio(" Timeframe",
    options= list( different_timeframes.keys()),
    horizontal= True,
    index=3)
#timeframe configuration
timeframes_config = different_timeframes[timeframe]


#default rsi window
rsi_window_default =window_rsi[timeframe]


#strategy selection
strategies_selected= st.sidebar.radio( " Strategies",["Buy & Hold", "SMA Crossover", "SMA + RSI", "Simple Momentum","Strategy Comparaison"], index=0)

strategies_using_sma = strategies_selected in ["SMA Crossover", "SMA + RSI"]

#default momentum parameter lookback
momentum_parameter_value = 20

# show momentum settings in the sidebar to choose how many past periods are used to compute momentum indicator
if strategies_selected == "Simple Momentum":
    with st.sidebar.expander(" Momentum parameters", expanded=True):
        momentum_parameter_value = st.slider(
            "Momentum lookback (number of periods)",
            min_value=5,
            max_value=200,
            value=20
        )

#get default SMA values based on the selected timeframe
sma_fast_default = sma_values[timeframe]["fast"]
sma_slow_default = sma_values[timeframe]["slow"]

# we show by default the SMA values and the checkbox to adjust them manually
if strategies_using_sma:
    with st.sidebar.expander(" Moving Average parameters", expanded=True):

        manual_sma = st.checkbox("Manual SMA")

        if manual_sma:
# Sliders for the fast and slow moving average
            sma_fast = st.slider(
                "Fast SMA",
                min_value=5,
                max_value=100,
                value=sma_fast_default
            )
            sma_slow = st.slider(
                "Slow SMA",
                min_value=20,
                max_value=300,
                value=sma_slow_default
            )
        else:
            sma_fast = sma_fast_default
            sma_slow = sma_slow_default

            st.caption(
                f"Automatic values for {timeframe} timeframe\n"
                f" Fast SMA: {sma_fast}\n"
                f" Slow SMA: {sma_slow}"
            )

else:
# SMA not used safe defaults
    sma_fast = sma_fast_default
    sma_slow = sma_slow_default

# the user can't put a fast sma upper than the slow sma
if strategies_using_sma and sma_fast >= sma_slow:
    st.warning(
       "Wrong SMA values.\n\n"
      "Fast SMA must be lower than Slow SMA."
    )
    st.stop()

# Default rsi parameters
rsi_window= 14
rsi_buy =55
rsi_sell= 70

# change and adapt treshold for rsi to signal wehn to buy and sell
if strategies_selected == "SMA + RSI":
    with st.sidebar.expander(" RSI parameters", expanded=True):
        rsi_window = st.slider(
            "RSI window",
            min_value= 5,
            max_value =30,
            value=rsi_window_default)
        rsi_buy =st.slider(
            "RSI buy threshold",
            min_value =30,
            max_value= 60,
            value =55 )
        rsi_sell = st.slider(
            "RSI sell threshold",
            min_value = 60,
            max_value=90,
            value=70)

#downloading market data
#we use a buffer to extend periods to compute indicators
# we use cache data to speed up the app
@st.cache_data
def download_data(ticker, period, interval):
    buffer = { "1d":"5d",
        "5d":"1mo",
        "1mo":"3mo",
        "6mo":"1y",
        "1y": "2y",
        "5y": "10y",
        "max": "max" }
    period_large = buffer.get(period, period)

#downloading data from yfinance
    return yf.download(
        ticker,
        period=period_large,
        interval=interval,
        progress=False
    )

# Show loading message
with st.spinner("Downloading data"):
    data=download_data(ticker,timeframes_config[ "period"], timeframes_config["interval"])
# Stop if no data
if data.empty:
    st.error("No data")
    st.stop()

# we keep only close prices and rename columns
price= data["Close"].reset_index()
price.columns = ["Datetime", "Close"]

#we take the last real closing price and the currency of the asset
last_close_raw= float(data["Close"].iloc[-1])
currency=yf.Ticker(ticker).info.get( "currency", "USD")

#header  content to display ticker with its current price and currency
st.markdown(
    f"## {ticker} — {last_close_raw:.2f} {currency}",
    unsafe_allow_html=False)

st.caption(
    f"Last update: {datetime.now(time_zone_france).strftime('%H:%M:%S')}"
)


#indicators

#compute percentage return of the price
price["return"]=price["Close"].pct_change()

# Compute fast and slow moving average
price["sma_fast"] = price["Close"].rolling(sma_fast).mean()
price["sma_slow"] = price["Close"].rolling(sma_slow).mean()

#computing RSI
def rsi(prices, window=14):
# we cmpute price changes and keep only positives for gains and negatives for losses
    price_change= prices.diff()
    gains= price_change.clip(lower=0)
    losses =-price_change.clip(upper=0)

# Compute average gain and loss to have the relative strength
    average_gain = gains.ewm(alpha=1/window, adjust=False).mean()
    average_loss = losses.ewm(alpha=1/window, adjust=False).mean()
    relative_stregth = average_gain / average_loss
#converting to relative strength index
    rsi_value = 100 - (100 / (1 + relative_stregth))
    return rsi_value

# Add RSI to the dataframe
price["rsi"]= rsi( price["Close"],rsi_window)


#trading strategies


# strategy 1 : SMA Crossover

#we create a position: we put 1 if the fast SMA is upper thant the slow one, else 0
position= np.where(price["sma_fast"] > price["sma_slow"], 1, 0)

# we shift the position to avoid data look ahead biais using futur informations
price["Position"]= pd.Series(position).shift(1).fillna(0)

#computing the strategy
price["SMA_Crossover"]= ( (1 + price["return"] * price["Position"]).fillna(1).cumprod() * capital)



# strategy 2 : Momentum

#computing momentum over the number of periods determined by the parameter
price["Momentum_computed"]=price["Close"].pct_change(periods=momentum_parameter_value)

# if momentum is positive, we put 1 for the position
# fixed momentum threshold
position_momentum = np.where(price["Momentum_computed"] > 0.005, 1, 0)

# we shift the position to avoid data look ahead biais using futur informations
price["Position_Momentum"]= pd.Series(position_momentum).shift(1).fillna(0)

# compute momentum strategy value
price["Simple_Momentum"]=( (1 + price["return"] * price["Position_Momentum"]).fillna(1).cumprod() * capital)



# strategy 3 : SMA + RSI

#creating a combined sgnal with these two indicators
signal_sma_rsi =np.where(( price["sma_fast"]> price["sma_slow"])& (price["rsi"] < rsi_buy ), 1,np.where(price["rsi"] > rsi_sell, 0, np.nan))

# build the final position
price[ "Position_SMA_RSI"]=( pd.Series(signal_sma_rsi).ffill().shift(1).fillna(0))

#compute Buy & Hold strategy
price["Buy_and_Hold"] = price["Close"] / price["Close"].iloc[0] * capital


# for the final data window, we keep only the selectect timeframe after the buffer
if timeframe in number_days:
    last_dt= price["Datetime"].max()
    start_dt=last_dt - pd.Timedelta(days=number_days[timeframe])
    price= price[price["Datetime"] >= start_dt].copy()

#we detect the buy and sell signals
price[ "Buy_Signal" ] =( (price["Position_SMA_RSI"] == 1) & (price["Position_SMA_RSI"].shift(1) ==0))
price["Sell_Signal"]=(( price["Position_SMA_RSI"]== 0) &( price["Position_SMA_RSI"].shift(1) == 1))

# compute SMA + RSI strategy value
price[ "SMA_RSI"] =( (1+ price["return"] *price["Position_SMA_RSI"]).fillna(1).cumprod()* capital)


#re-normalize all strategies at window start
for col in ["Buy_and_Hold", "SMA_Crossover", "SMA_RSI", "Simple_Momentum"]:
    price[col] = price[col] / price[col].iloc[0] * capital


# metrics

# compute maximum drawdown
def max_drawdown(series):
#difference between value and historical maximum
    drawdown=( series -series.cummax()) / series.cummax()
    return drawdown.min()

#annualization factors for different data frequencies
annual_factor= {"5m": 252*78, "1h": 252*7, "1d": 252}
factor= np.sqrt(annual_factor.get(timeframes_config["interval"], 252))


# compute sharpe ratio
def sharpe(returns):
#avoid division by zero
    if returns.std() == 0:
        return np.nan
    sharpe_value =returns.mean() / returns.std() * factor
    return sharpe_value

# compute  volatility
def volatility(returns):
#avoid division by zero
    if returns.std() == 0:
        return np.nan
# volatility annualized using the same factor as Sharpe
    volatility_value = returns.std() * factor
    return volatility_value

#create a table to store performance and risk metrics for each strategy
metrics = pd.DataFrame({

# we put 4 different metrics for each strategy : Total return , sharpe ratio, volatility and maximum drawdown
    "Buy & Hold": [
        price["Buy_and_Hold"].iloc[-1] / capital - 1,
        sharpe(price["return"]),
        volatility(price["return"]),
        max_drawdown(price["Buy_and_Hold"])
    ],

    "SMA Crossover": [
        price["SMA_Crossover"].iloc[-1] / capital - 1,
        sharpe(price["return"] * price["Position"]),
        volatility(price["return"] * price["Position"]),
        max_drawdown(price["SMA_Crossover"])
    ],

    "Simple Momentum": [
        price["Simple_Momentum"].iloc[-1] / capital - 1,
        sharpe(price["return"] * price["Position_Momentum"]),
        volatility(price["return"] * price["Position_Momentum"]),
        max_drawdown(price["Simple_Momentum"])
    ],

    "SMA + RSI": [
        price["SMA_RSI"].iloc[-1] / capital - 1,
        sharpe(price["return"] * price["Position_SMA_RSI"]),
        volatility(price["return"] * price["Position_SMA_RSI"]),
        max_drawdown(price["SMA_RSI"])
    ]

},
# names of the rows for performance and risk metrics
index=[
    "Total Return",
    "Sharpe Ratio",
    "Volatility",
    "Max Drawdown"
]
).round(3)

performance_metrics= metrics.loc[ ["Total Return", "Max Drawdown"]].copy()
risk_metrics= metrics.loc[ ["Sharpe Ratio", "Volatility"]].copy()

# Final portfolio values
final_values = pd.DataFrame({
    "Final Value ($)": [
        price["Buy_and_Hold"].iloc[-1],
        price["SMA_Crossover"].iloc[-1],
        price["SMA_RSI"].iloc[-1],
        price["Simple_Momentum"].iloc[-1]
    ]
}, index=[
    "Buy & Hold",
    "SMA Crossover",
    "SMA + RSI",
    "Simple Momentum"
]).round(2)


# display charts and tables

#we create a figure to compare all strategies on the same chart
if strategies_selected == "Strategy Comparaison":

    st.subheader("Strategy Comparaison")

    fig, ax = plt.subplots(figsize=(13, 6))
    ax.plot(price["Datetime"],price["Buy_and_Hold"], label="Buy & Hold", linewidth=2)
    ax.plot(price["Datetime"], price["SMA_Crossover"], label="SMA Crossover")
    ax.plot(price["Datetime"],price["SMA_RSI"], label="SMA + RSI")
    ax.plot(price["Datetime"],price["Simple_Momentum"], label="Simple Momentum")
    ax.axhline(capital, linestyle="--", color="gray", alpha=0.6)

    ax.set_ylabel("Portfolio Value ($)")
    ax.grid(alpha=0.3)
    ax.legend()
    st.pyplot(fig)

# Display performance and risk metrics table
    st.markdown("### Performance Metrics")
    st.dataframe(performance_metrics.style.format("{:.2%}"), use_container_width=True)
    st.markdown("### Risk Metrics")
    st.dataframe(risk_metrics.style.format("{:.2f}"), use_container_width=True)
#display final portfolio values
    st.markdown("### Final Portfolio Values")
    st.dataframe(final_values, use_container_width=True)


# we display one strategy at a time, with charts on left and tables on right
else:
    selected_col = strategies_labels[strategies_selected]
    charts, tables = st.columns([2, 1])


    with charts:
# we show charts for the buy and hold strategy: price and portfolio value
        if strategies_selected == "Buy & Hold":
            fig, (ax1, ax2) = plt.subplots(
                2, 1, figsize=(12, 7), sharex=True,
                gridspec_kw={"height_ratios": [3, 1]}
            )

            ax1.plot(price["Datetime"], price["Close"], color="black")
            ax1.set_title("Asset price")
            ax1.grid(alpha=0.3)
            ax2.plot(price["Datetime"], price["Buy_and_Hold"], color="green")
            ax2.axhline(capital, linestyle="--", color="gray", alpha=0.6)
            ax2.set_title("Buy & Hold performance")
            ax2.grid(alpha=0.3)
            st.pyplot(fig)

#we display price with moving average and the performance of the sma crossover strategy
        elif strategies_selected == "SMA Crossover":
            fig, (ax1, ax2) = plt.subplots(
                2, 1, figsize=(12, 7), sharex=True,
                gridspec_kw={"height_ratios": [3, 1]}
            )
            ax1.plot(price["Datetime"], price["Close"], color="black", label="Price")
            ax1.plot(price["Datetime"], price["sma_fast"], "--", label="Fast SMA")
            ax1.plot(price["Datetime"], price["sma_slow"], "--", label="Slow SMA")
            ax1.set_title("Price and SMAs")
            ax1.grid(alpha=0.3)
            ax1.legend()
            ax2.plot(price["Datetime"], price["SMA_Crossover"], color="green")
            ax2.axhline(capital, linestyle="--", color="gray", alpha=0.6)
            ax2.set_title("SMA Crossover performance")
            ax2.grid(alpha=0.3)
            st.pyplot(fig)

#we show the price, the RSI indicator and the performance of this strategy
        elif strategies_selected == "SMA + RSI":
            fig, (ax1, ax2, ax3) = plt.subplots(
                3, 1, figsize=(12, 9), sharex=True,
                gridspec_kw={"height_ratios": [3, 1, 1]}
            )

            ax1.plot(price["Datetime"], price["Close"], color="black", label="Price")
            ax1.plot(price["Datetime"], price["sma_fast"], "--", label="Fast SMA")
            ax1.plot(price["Datetime"], price["sma_slow"], "--", label="Slow SMA")

            ax1.scatter(
                price.loc[price["Buy_Signal"], "Datetime"],
                price.loc[price["Buy_Signal"], "Close"],
                marker="^", color="green", s=80, label="Buy"
            )
            ax1.scatter(
                price.loc[price["Sell_Signal"], "Datetime"],
                price.loc[price["Sell_Signal"], "Close"],
                marker="v", color="red", s=80, label="Sell"
            )

            ax1.set_title("Price + SMAs + Buy/Sell signals")
            ax1.grid(alpha=0.3)
            ax1.legend()

            ax2.plot(price["Datetime"], price["rsi"], color="purple")
            ax2.axhline(rsi_buy, linestyle="--", color="green")
            ax2.axhline(rsi_sell, linestyle="--", color="red")
            ax2.set_ylim(0, 100)
            ax2.set_title("RSI")
            ax2.grid(alpha=0.3)

            ax3.plot(price["Datetime"], price["SMA_RSI"], color="green")
            ax3.axhline(capital, linestyle="--", color="gray", alpha=0.6)
            ax3.set_title("SMA + RSI performance")
            ax3.grid(alpha=0.3)
            st.pyplot(fig)

#we show the price, the momentum indicator and the performance of this strategy
        elif strategies_selected == "Simple Momentum":
            fig, (ax1, ax2, ax3) = plt.subplots(
                3, 1, figsize=(12, 9), sharex=True,
                gridspec_kw={"height_ratios": [3, 1, 1]}
            )

            ax1.plot(price["Datetime"], price["Close"], color="black")
            ax1.set_title("Price")
            ax1.grid(alpha=0.3)

            ax2.plot(price["Datetime"], price["Momentum_computed"], color="orange")
            ax2.axhline(0, linestyle="--", color="gray")
            ax2.set_title("Momentum")
            ax2.grid(alpha=0.3)

            ax3.plot(price["Datetime"], price["Simple_Momentum"], color="green")
            ax3.axhline(capital, linestyle="--", color="gray", alpha=0.6)
            ax3.set_title("Momentum strategy performance")
            ax3.grid(alpha=0.3)

            st.pyplot(fig)
#we display tables with performance and risk metrics, aznd the final portfolio value
    with tables:
        st.subheader("Performance")
        st.dataframe(
            performance_metrics[[selected_col]].style.format("{:.2%}"),
            use_container_width=True
        )

        st.subheader("Risk")
        st.dataframe(
            risk_metrics[[selected_col]].style.format("{:.2f}"),
            use_container_width=True
        )

        st.subheader("Final Value")
        st.dataframe(
            final_values.loc[[selected_col]],
            use_container_width=True
        )




Overwriting quantA.py


In [3]:
!pkill -f streamlit
!pkill -f ngrok


In [4]:
!rm -f logs.txt


In [5]:
!streamlit run quantA.py --server.port 8501 --server.headless true &>/content/logs.txt &


In [6]:
!ngrok config add-authtoken 37uEKVfQevKRGnP7Dj37eymdP2x_7wyZ5wwhurixpkYhwuy6j


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
from pyngrok import ngrok

ngrok.kill()
public_url = ngrok.connect(8501)
print(" Open this link:", public_url)


 Open this link: NgrokTunnel: "https://unobserved-daubingly-nohemi.ngrok-free.dev" -> "http://localhost:8501"
